In [1]:
%reload_ext autoreload
%autoreload 2

"""
This script runs the bootstrap kfold validation experiments as used in
the publication.

Usage:
  validation.py [--interpro] [--pfam] [--mf] [--cc] [--bp]
             [--use_cache] [--induce] [--verbose]
             [--model=M] [--n_jobs=J] [--n_splits=S] [--n_iterations=I]
             [--directory=DIR]
  validation.py -h | --help

Options:
  -h --help     Show this screen.
  --interpro    Use interpro domains in features.
  --pfam        Use Pfam domains in features.
  --mf          Use Molecular Function Gene Ontology in features.
  --cc          Use Cellular Compartment Gene Ontology in features.
  --bp          Use Biological Process Gene Ontology in features.
  --induce      Use ULCA inducer over Gene Ontology.
  --verbose     Print intermediate output for debugging.
  --use_cache   Use cached features if available.
  --model=M         A binary classifier from Scikit-Learn implementing fit,
                    predict and predict_proba [default: LogisticRegression]
  --n_jobs=J        Number of processes to run in parallel [default: 1]
  --n_splits=S      Number of cross-validation splits [default: 5]
  --n_iterations=I  Number of bootstrap iterations [default: 5]
  --directory=DIR   Output directory [default: ./results/]
"""

import json
import logging
import pandas as pd
import numpy as np
from operator import itemgetter
from collections import Counter
from datetime import datetime

from pyppi.base import parse_args, su_make_dir
from pyppi.data import load_network_from_path, load_ptm_labels
from pyppi.data import testing_network_path, training_network_path
from pyppi.data import load_go_dag, ipr_name_map, pfam_name_map

from pyppi.models.binary_relevance import BinaryRelevance
from pyppi.models import make_classifier
from pyppi.model_selection.scoring import MultilabelScorer, Statistics
from pyppi.model_selection.experiment import KFoldExperiment, Bootstrap
from pyppi.model_selection.sampling import IterativeStratifiedKFold

from pyppi.data_mining.features import AnnotationExtractor
from pyppi.data_mining.uniprot import UniProt, get_active_instance
from pyppi.data_mining.tools import xy_from_interaction_frame

from pyppi.data_mining.ontology import id_to_node

from sklearn.base import clone
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import (
    recall_score, make_scorer, 
    label_ranking_average_precision_score,
    label_ranking_loss
)

from sklearn.datasets import make_multilabel_classification

logging.basicConfig(
    format='[%(asctime)s] %(levelname)s: %(message)s', 
    datefmt='%m-%d-%Y %I:%M:%S',
    level=logging.DEBUG,
)
logger = logging.getLogger(__name__)

args = {
    'n_jobs': 1,
    'n_splits': 5,
    'n_iterations': 5,
    'induce': True,
    'verbose': True,
    'selection': [
        UniProt.data_types().GO_MF.value,
        UniProt.data_types().GO_BP.value,
        UniProt.data_types().GO_CC.value,
        UniProt.data_types().INTERPRO.value,
        UniProt.data_types().PFAM.value
    ],
    'model': 'LogisticRegression',
    'use_cache': True,
    'directory': './results/'
}
n_jobs = args['n_jobs']
n_splits = args['n_splits']
n_iter = args['n_iterations']
induce = args['induce']
verbose = args['verbose']
selection = args['selection']
model = args['model']
use_feature_cache = args['use_cache']
direc = args['directory']
backend = 'multiprocessing'
get_top_n = 25
go_dag = None

# Set up the folder for each experiment run named after the current time
folder = datetime.now().strftime("val_%y-%m-%d_%H-%M")
direc = "{}/{}/".format(direc, folder)
su_make_dir(direc)
json.dump(
    args, fp=open("{}/settings.json".format(direc), 'w'),
    indent=4, sort_keys=True)

load obo file C:\Users\Daniel\.pyppi\go.obo
C:\Users\Daniel\.pyppi\go.obo: fmt(1.2) rel(2017-07-14) 48,971 GO Terms


In [2]:
logging.info("Loading training and testing data.")
if not go_dag:
    go_dag = load_go_dag()
ipr_map = ipr_name_map(lowercase_keys=False)
pfam_map = pfam_name_map(lowercase_keys=False)
uniprot = get_active_instance(
    verbose=verbose,
    sprot_cache=None,
    trembl_cache=None
)
data_types = UniProt.data_types()
labels = load_ptm_labels()
annotation_ex = AnnotationExtractor(
    induce=induce,
    selection=selection,
    n_jobs=n_jobs,
    verbose=verbose,
    cache=use_feature_cache,
    backend='multiprocessing'
)
training = load_network_from_path(training_network_path)
testing = load_network_from_path(testing_network_path)

[08-17-2017 03:42:42] INFO: Loading training and testing data.


load obo file C:\Users\Daniel\.pyppi\go.obo
C:\Users\Daniel\.pyppi\go.obo: fmt(1.2) rel(2017-07-14) 48,971 GO Terms
First time loading on UniProt instance. Make take a few moments


In [3]:
# Get the features into X, and multilabel y indicator format
logging.info("Preparing training and testing data.")
mlb = MultiLabelBinarizer(classes=labels)
X_train_ppis, y_train = xy_from_interaction_frame(training)
X_test_ppis, y_test = xy_from_interaction_frame(testing)
mlb.fit(y_train)

logging.info("Computing class distributions.")
json.dump(
    Counter([l for ls in y_train for l in ls]), 
    fp=open("{}/training_distribution.json".format(direc), 'w'),
    indent=4, sort_keys=True
)
json.dump(
    Counter([l for ls in y_test for l in ls]), 
    fp=open("{}/testing_distribution.json".format(direc), 'w'),
    indent=4, sort_keys=True
)

X_train = annotation_ex.transform(X_train_ppis)
X_test = annotation_ex.transform(X_test_ppis)
y_train = mlb.transform(y_train)
y_test = mlb.transform(y_test)

[08-17-2017 03:43:34] INFO: Preparing training and testing data.
[08-17-2017 03:43:34] INFO: Computing class distributions.


Finding new PPIs...
Stringing selected features for each PPI...
Finding new PPIs...
Stringing selected features for each PPI...


In [4]:
def specificity(y_true, y_pred):
    y_true = np.asarray(list(y_true))
    y_pred = np.asarray(list(y_pred))
    tn = np.sum(y_true == 0)
    fp = np.sum([True for (a, b) in zip(y_true, y_pred) if a == 0 and b == 1])
    if (tn + fp) == 0:
        return np.NaN
    return tn / (tn + fp)


def get_coefs(clf):
    """
    Return the feature weightings for each estimator. If estimator is a
    pipeline, then it assumes the last step is the estimator.

    :return: array-like, shape (n_classes_, n_features)
    """
    def feature_imp(estimator):
        if hasattr(estimator, 'steps'):
            estimator = estimator.steps[-1][-1]
        if hasattr(estimator, "coef_"):
            return estimator.coef_
        elif hasattr(estimator, "coefs_"):
            return estimator.coefs_
        elif hasattr(estimator, "feature_importances_"):
            return estimator.feature_importances_
        else:
            raise AttributeError(
                "Estimator {} doesn't support "
                "feature coefficients.".format(type(estimator)))

    return feature_imp(clf.best_estimator_)


def top_n_features(n, clf, absolute=True, vectorizer=None):
    """
    Return the top N features. If clf is a pipeline, then it assumes
    the first step is the vectoriser holding the feature names.

    :return: array like, shape (n_estimators, n).
        Each element in a list is a tuple (feature_idx, weight).
    """
    top_features = []
    coefs = get_coefs(clf)[0]

    if absolute:
        coefs = abs(coefs)
    if hasattr(clf, 'steps') and vectorizer is None:
        vectorizer = clf.steps[0][-1]
    idx_coefs = sorted(
        enumerate(coefs), key=itemgetter(1), reverse=True
    )[:n]
    if vectorizer:
        idx = [idx for (idx, w) in idx_coefs]
        ws = [w for (idx, w) in idx_coefs]
        names = np.asarray(vectorizer.get_feature_names())[idx]
        return list(zip(names, ws))
    else:
        return idx_coefs
    
def get_term_name(go_dag, term):
    if 'go' in term.lower():
        term = term.replace("go", "GO:")
        return go_dag[term.upper()].name
    elif 'ipr' in term.lower():
        return ipr_map[term.upper()]
    elif 'pf' in term.lower():
        return pfam_map[term.upper()]
    return None
        

In [5]:
logging.info("Setting up preliminaries and the statistics arrays")
logging.info("Found classes {}".format(', '.join(mlb.classes)))
n_classes = len(mlb.classes)
seeds = range(n_iter)
top_features = {l:[] for l in labels}
param_distribution = {
    'C': np.arange(0.01, 20.01, step=0.01),
    'penalty': ['l1', 'l2']
}

binary_scoring_funcs = [
    ('Binary F1', f1_score) , 
    ('Precision', precision_score), 
    ('Recall', recall_score),
    ('Specificity', recall_score)
]
multilabel_scores_funcs = [
    ('Label Ranking Loss', label_ranking_loss), 
    ('Label Ranking Average Precision', label_ranking_average_precision_score), 
    ('Macro (weighted) F1', f1_score), 
    ('Macro (un-weighted) F1', f1_score)
]
n_scorers = len(binary_scoring_funcs)
n_ml_scorers = len(multilabel_scores_funcs)

# 2: position 0 is for validation, position 1 is for testing
binary_statistics = np.zeros((n_iter, n_splits, n_classes, 2, n_scorers))
multilabel_statistics = np.zeros((n_iter, n_splits, 2, n_ml_scorers))

[08-17-2017 03:45:44] INFO: Setting up preliminaries and the statistics arrays
[08-17-2017 03:45:44] INFO: Found classes carboxylation, methylation, dephosphorylation, activation, sulfation, phosphorylation, deacetylation, prenylation, ubiquitination, proteolytic-cleavage, binding/association, acetylation, glycosylation, inhibition, state-change, dissociation, myristoylation, sumoylation


In [6]:
for bs_iter in range(n_iter):
    logging.info("Fitting bootstrap iteration {}.".format(bs_iter + 1))
    cv = IterativeStratifiedKFold(n_splits=n_splits, random_state=seeds[bs_iter])
    
    for fold_iter, (train_idx, validation_idx) in enumerate(cv.split(X_train, y_train)):
        logging.info("Fitting fold iteration {}.".format(fold_iter + 1))
        y_valid_f_pred = []
        y_test_f_pred = []
        y_valid_f_proba = []
        y_test_f_proba = []
        
        for label_idx, label in enumerate(labels):
            logging.info("Fitting label {}.".format(label))
            
            # Prepare all training and testing data
            logging.info("Preparing data.")
            vectorizer = CountVectorizer(binary=False)
            vectorizer.fit(X_train[train_idx])

            X_train_l = vectorizer.transform(X_train[train_idx])
            y_train_l = y_train[train_idx, label_idx]
        
            X_valid_l = vectorizer.transform(X_train[validation_idx])
            y_valid_l = y_train[validation_idx, label_idx]

            X_test_l = vectorizer.transform(X_test)
            y_test_l = y_test[:, label_idx]
            
            # Build and fit classifier
            logging.info("Fitting classifier.")
            clf = RandomizedSearchCV(
                estimator=make_classifier(algorithm=model, random_state=0),
                scoring='f1', cv=3, n_iter=1, n_jobs=n_jobs, refit=True,
                random_state=0, param_distributions=param_distribution,
            )
            clf.fit(X_train_l, y_train_l)
            
            # Validation scores in binary and probability format
            y_valid_l_pred = clf.predict(X_valid_l)
            y_valid_l_proba = clf.predict_proba(X_valid_l)
            
            # Held-out testing scores in binary and probability format
            y_test_l_pred = clf.predict(X_test_l)
            y_test_l_proba = clf.predict_proba(X_test_l)
            
            # Store these per label results in a list which we will
            # later use to stack into a multi-label array.
            y_valid_f_pred.append([[x] for x in y_valid_l_pred])
            y_valid_f_proba.append([[x[1]] for x in y_valid_l_proba])
            
            y_test_f_pred.append([[x] for x in y_test_l_pred])
            y_test_f_proba.append([[x[1]] for x in y_test_l_proba])
            
            # Perform scoring on the validation set and the external testing set.
            logging.info("Computing fold label binary performance.")
            for func_idx, (_, func) in enumerate(binary_scoring_funcs):
                scores_v = func(y_valid_l, y_valid_l_pred, average='binary')
                scores_t = func(y_test_l, y_test_l_pred, average='binary')
                binary_statistics[bs_iter, fold_iter, label_idx, 0, func_idx] = scores_v
                binary_statistics[bs_iter, fold_iter, label_idx, 1, func_idx] = scores_t
                
            logging.info("Computing top label features for fold.")
            # Get the top 20 features for this labels's run.
            top_n = [
                f for f, _ in 
                top_n_features(
                    clf=clf, n=get_top_n, absolute=True, vectorizer=vectorizer)
            ]
            top_features[label].extend(top_n)
            break
        
        logging.info("Computing fold mult-label performance.")
        # True scores in multi-label indicator format
        y_valid_f = y_train[validation_idx]
        y_test_f = y_test
        
        # Validation scores in multi-label indicator format
        y_valid_f_pred = np.hstack(y_valid_f_pred)
        y_valid_f_proba = np.hstack(y_valid_f_proba)
        
        # Testing scores in multi-label probability format
        y_test_f_pred = np.hstack(y_test_f_pred)
        y_test_f_proba = np.hstack(y_test_f_proba)
        
        for func_idx, (func_name, func) in enumerate(multilabel_scores_funcs):
            if func_name == 'Macro (weighted) F1':
                scores_v = func(y_valid_f, y_valid_f_pred, average='weighted')
                scores_t = func(y_test_f, y_test_f_pred, average='weighted')
            elif func_name == 'Macro (un-weighted) F1':
                scores_v = func(y_valid_f, y_valid_f_pred, average='macro')
                scores_t = func(y_test_f, y_test_f_pred, average='macro')
            elif func_name == 'Label Ranking Average Precision':
                scores_v = func(y_valid_f, y_valid_f_proba)
                scores_t = func(y_test_f, y_test_f_proba)
            else:
                scores_v = func(y_valid_f, y_valid_f_pred)
                scores_t = func(y_test_f, y_test_f_pred)
                
            multilabel_statistics[bs_iter, fold_iter, 0, func_idx] = scores_v
            multilabel_statistics[bs_iter, fold_iter, 1, func_idx] = scores_t
        break

[08-17-2017 03:45:45] INFO: Fitting bootstrap iteration 1.
[08-17-2017 03:45:48] INFO: Fitting fold iteration 1.
[08-17-2017 03:45:48] INFO: Fitting label carboxylation.
[08-17-2017 03:45:48] INFO: Preparing data.
[08-17-2017 03:45:58] INFO: Fitting classifier.
[08-17-2017 03:46:01] INFO: Computing fold label binary performance.
[08-17-2017 03:46:01] INFO: Computing top label features for fold.
[08-17-2017 03:46:01] INFO: Computing fold mult-label performance.


ValueError: y_true and y_score have different shape

In [ ]:
logging.info("Writing statistics to file.")
func_names = [n for n, _ in binary_scoring_funcs]
iterables = [range(n_iter), range(n_splits), range(n_classes), ["validation", "holdout"], func_names]
names=['bootstrap iteration', 'fold iteration', 'labels', 'condition', 'score function']
index = pd.MultiIndex.from_product(iterables, names=names)
binary_df = pd.DataFrame(binary_statistics.ravel(), index=index)
binary_df.to_csv('{}/{}'.format(folder, 'binary_stats.csv'), sep=',', index=False)

func_names = [n for n, _ in multilabel_scores_funcs]
iterables = [range(n_iter), range(n_splits), ["validation", "holdout"], func_names]
names=['bootstrap iteration', 'fold iteration', 'condition', 'score function']
index = pd.MultiIndex.from_product(iterables, names=names)
multilabel_df = pd.DataFrame(multilabel_statistics.ravel(), index=index)
multilabel_df.to_csv('{}/{}'.format(folder, 'multilabel_stats.csv'), sep=',', index=False)

In [14]:
logging.info("Computing top label features overall.")
iterables = [mlb.classes, range(get_top_n), range(1)]
names=['label', 'name', 'description']
index = pd.MultiIndex.from_product(iterables, names=names)
data = np.zeros(shape=(len(mlb.classes), get_top_n, 1), dtype='object')

for i, (label, feature_ls) in enumerate([(l, top_features[l]) for l in mlb.classes]):
    counts = Counter(feature_ls)
    counts = sorted(counts.items(), key=itemgetter(1), reverse=True)
    top_n_terms = [term for term, _ in counts][:get_top_n]
    top_n_desc = [get_term_name(go_dag, t) for t in top_n_terms]
    
    for j, (term, desc) in enumerate(zip(top_n_terms, top_n_desc)):
        data[i, j, 0] = [term, desc]
        print(data[i])

    
logging.info("Writing top features to file.")
top_features_df = pd.DataFrame(data=data.ravel(), index=index)   
top_features_df.to_csv('{}/{}'.format(folder, 'top_features.csv'), sep=',', index=False)

[08-17-2017 03:51:52] INFO: Computing top label features overall.


[[list(['go0005886', 'plasma membrane'])]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[list(['go0005886', 'plasma membrane'])]
 [list(['go0000002', 'mitochondrial genome maintenance'])]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[list(['go0005886', 'plasma membrane'])]
 [list(['go0000002', 'mitochondrial genome maintenance'])]
 [list(['go0000014', 'single-stranded DNA endodeoxyribonuclease activity'])]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[list(['go0005886', 'plasma membrane'])]
 [list(['go0000002', 'mitochondrial genome maintenance'])]
 [list(['go0000014', 'single-stranded DNA endodeoxyribonuclease activity'])]
 [list(['go0000019', 'regulation of mitotic recombination'])]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 

 [0]]
[[list(['go0005886', 'plasma membrane'])]
 [list(['go0000002', 'mitochondrial genome maintenance'])]
 [list(['go0000014', 'single-stranded DNA endodeoxyribonuclease activity'])]
 [list(['go0000019', 'regulation of mitotic recombination'])]
 [list(['go0005515', 'protein binding'])]
 [list(['go0008488', 'gamma-glutamyl carboxylase activity'])]
 [list(['go0000041', 'transition metal ion transport'])]
 [list(['go0003824', 'catalytic activity'])]
 [list(['go0000011', 'vacuole inheritance'])]
 [list(['ipr014710', 'RMLC-LIKE_JELLYROLL'])]
 [list(['ipr011020', 'HTTM'])]
 [list(['go0016831', 'carboxy-lyase activity'])]
 [list(['go0000035', 'acyl binding'])]
 [list(['go0000012', 'single strand break repair'])]
 [list(['go0043231', 'intracellular membrane-bounded organelle'])]
 [list(['go0018200', 'peptidyl-glutamic acid modification'])]
 [list(['go0016830', 'carbon-carbon lyase activity'])]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[list(['go0005886', 'plasma membrane'])]
 [list(['go000000

 [0]]
[[list(['go0005886', 'plasma membrane'])]
 [list(['go0000002', 'mitochondrial genome maintenance'])]
 [list(['go0000014', 'single-stranded DNA endodeoxyribonuclease activity'])]
 [list(['go0000019', 'regulation of mitotic recombination'])]
 [list(['go0005515', 'protein binding'])]
 [list(['go0008488', 'gamma-glutamyl carboxylase activity'])]
 [list(['go0000041', 'transition metal ion transport'])]
 [list(['go0003824', 'catalytic activity'])]
 [list(['go0000011', 'vacuole inheritance'])]
 [list(['ipr014710', 'RMLC-LIKE_JELLYROLL'])]
 [list(['ipr011020', 'HTTM'])]
 [list(['go0016831', 'carboxy-lyase activity'])]
 [list(['go0000035', 'acyl binding'])]
 [list(['go0000012', 'single strand break repair'])]
 [list(['go0043231', 'intracellular membrane-bounded organelle'])]
 [list(['go0018200', 'peptidyl-glutamic acid modification'])]
 [list(['go0016830', 'carbon-carbon lyase activity'])]
 [list(['ipr011051', 'RMLC_CUPIN'])]
 [list(['go0007165', 'signal transduction'])]
 [list(['go004687

[08-17-2017 03:51:52] INFO: Writing top features to file.


FileNotFoundError: [Errno 2] No such file or directory: 'val_17-08-17_15-42/top_features.csv'

In [ ]:
top_features_df.index